In [ ]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import Session
import json
import sagemaker
import yt_dlp
from moviepy.video.io.VideoFileClip import VideoFileClip
import boto3
import os

In [ ]:
session = Session()
instance_type = "ml.g5.2xlarge"
bucket = 'slip-ml'

In [ ]:
image_uri = sagemaker.image_uris.retrieve(framework='pytorch',
                             region=session.boto_region_name,
                             instance_type=instance_type,
                             image_scope='training')
print(image_uri)

In [ ]:
model = PyTorchModel(
    model_data="s3://slip-ml/models/llama_model.tar.gz",  # Dummy, as models are loaded from S3 in serve.py
    role="arn:aws:iam::438465160412:role/SageMakerRole",
    entry_point="serve.py",
    source_dir="source",
    image_uri=image_uri,
    sagemaker_session=session,
    env={
        "SM_DEFAULT_BUCKET": f"{bucket}",
        "VALLR_S3_KEY": "s3://sagemaker-us-east-1-438465160412/pytorch-training-2025-05-13-18-29-30-872/output/model.tar.gz",
        "LLAMA_S3_KEY": "models/llama_model.tar.gz",
    }
)

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,  # Single-GPU
    endpoint_name="vallr-llama-endpoint"
)

In [ ]:
def download_youtube_video_yt_dlp(url):
    # extract video ID from the URL
    video_id = url.split("v=")[-1]
    if "&" in video_id:
        video_id = video_id.split("&")[0]

    ydl_opts = {
        "outtmpl": f"{video_id}.%(ext)s",  # Output path and filename
        "format": "best",  # Select the best single file (video + audio)
        "merge_output_format": None,  # Avoid merging, stick to single stream
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print("Download completed successfully!")
        return f"{video_id}.mp4"
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
youtube_test = 'https://www.youtube.com/watch?v=ePRrIb1NCkw'
video_file = download_youtube_video_yt_dlp(youtube_test)
video = VideoFileClip(video_file)
chunk = video.subclipped(30, 40)
chunk_filename = f"testing_vid.mp4"
chunk.write_videofile(chunk_filename, codec="libx264")
# upload the video to S3
s3 = boto3.client('s3')
s3.upload_file(chunk_filename, bucket, chunk_filename)
# delete the local video file
os.remove(video_file)

In [ ]:
response = predictor.predict({
    "input_s3_path": f"s3://{bucket}/{chunk_filename}",
}, accept="application/json")
print(json.loads(response))